# __Homework 4:__ Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

---
The __actual sequences__ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
---


__The end of NA is__ `...CACGATAGATAAATAATAGTGCACCAT`
    
__The end of HA is__ `...CCGGATTTGCATATAATGATGCACCAT`

---    

    
The __sequencing reads__ from the reverse end of the molecules (in 5'>3' orientation), so the sequencing reads are as follows:

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of HA]-3'
or

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of NA]-3'

---   
    
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

# this is Remi's homework! 
## I worked with Jonathan Mattingly to debug our answers but all the work below is mine

In [46]:
#load the necessary packages
import re
import Bio.SeqIO
from Bio.Seq import Seq


In [47]:
#load data:
#to run this notebook, script and fastq file should be in same working directory
    #this was adapted from example code in lecture 9

#read the data as a list 
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))

In [48]:
#get just the sequence info out of the SeqRecord object

#first, initialize an empty list
seqreads_Seq = []

#get just the sequence without annotations out of seqreads
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)
#remember that at this point, each index of the list is a Seq object


In [49]:
#now that we have our data in a list of Seq objects, lets find the reverse complement of each
#note that this "reverse complement" is the actual sequence

#initialize an empty list
seqreads_RevComp = []

#assign each seq's reverse complement to the new list
for seq in seqreads_Seq:
    seqreads_RevComp.append(seq.reverse_complement())

In [50]:
#new function! :)
#this function will search each sequence read for the ending seq, which we can later check if they match NA or HA

def read_type(seqread, endlen=27, downstream='AGGCGGCCGC'):
    """Identify ending sequence of reads with known downstream sequence.
    
    Parameters
    ----------
    seqread : Seq object
        Nucleotide sequence of influenza reads.
    endlen : int
        Length of HA/NA end type defining sequence.
    downstream: str
        Sequence downstream of the defining HA/NA sequences.
        
    Returns
    -------
    str or None
        Sequence of the barcode in the forward orientation, or `None` if no match to expected barcoded sequence.
        
   #downstream seq: AGGCGGCCGC
        
    """
    
    #first convert seq object to str
    seqread_str = str(seqread)
    
    #assemble regular expression to look for a sequence of any characters, length 'endlen', before the 'downstream' sequence
    type_re = re.compile("(?P<type>.{" + str(endlen) + "})" + downstream)
    
    #store the matching strings
    match = type_re.search(seqread_str)

    #if you got a match, assign that string (which we called "type") to a new str variable "type"
    if match:
        type = match.group("type")
    else:
        type = None
    
    
    
    return type

In [51]:
# also re-define the same "read barcode" function from lecture 9; almost exactly the same
# we will use this if the read matches HA or NA to store its barcode

def read_barcode(seqread, bclen = 16, upstream='AGGCGGCCGC'):
    """Identify barcode with known upstream sequence.
    
    Parameters
    ----------
    seqread : Seq object
        Nucleotide sequence read matching UPSTREAM-BARCODE in reverse orientation.
    bclen : int
        Length of barcode
    upstream: str
        Sequence upstream of the barcode.
        
    Returns
    -------
    str or None
        Sequence of the barcode in the forward orientation, or `None` if no match to expected barcoded sequence.

        
    """

    sequence_str = str(seqread)
    
    barcode_re = re.compile(upstream + "(?P<barcode>[ATCG]{" + str(bclen) + "})$")
    
    match = barcode_re.search(sequence_str)
    if match:
        barcode = match.group("barcode")
    else:
        barcode = None
    
    return barcode

2. How many HA sequences did not have a valid barcode? Also anwer the same question for NA.

In [52]:

#initialize empty dictionaries for number of NA and HA reads and barcodes
type_counts = {'invalid':0,'NA':0,'HA':0}
NA_barcode_counts = {'invalid':0}
HA_barcode_counts = {'invalid':0}

#define sequences
NA = 'CACGATAGATAAATAATAGTGCACCAT'
HA = 'CCGGATTTGCATATAATGATGCACCAT'


#loop through each sequence read one-by-one
for seq in seqreads_RevComp:
    type = read_type(seq, endlen=27)

    #check if the end sequence is the NA defining seq
    if type == NA:
        type_counts['NA'] += 1   
        #if it's an NA sequence, get its barcode and count it in the NA dictionary
        barcode = read_barcode(seq, bclen=16)
        if barcode:
            if barcode in NA_barcode_counts:
                NA_barcode_counts[barcode] += 1
            else:
                NA_barcode_counts[barcode] = 1
        else:
            NA_barcode_counts['invalid'] += 1

    #check if the end sequence is the HA defining seq
    elif type == HA:
        type_counts['HA'] += 1

        #if it's an HA sequence, get its barcode and count it in the HA dictionary
        barcode = read_barcode(seq, bclen=16)
        if barcode:
            if barcode in HA_barcode_counts:
                HA_barcode_counts[barcode] += 1
            else:
                HA_barcode_counts[barcode] = 1
        else:
            HA_barcode_counts['invalid'] += 1
    
    #if doesnt 
    else:
        type_counts['invalid'] += 1

print(f"Found {type_counts['invalid']} invalid reads, {type_counts['NA']} NA reads, and {type_counts['HA']} HA reads ")
print(NA_barcode_counts)
print(HA_barcode_counts)



Found 711 invalid reads, 3973 NA reads, and 5316 HA reads 
{'invalid': 66, 'ACTAAATAACTTAAGC': 62, 'CTACCCGTTTCCCAAC': 118, 'CGTCTTCCATCCCCAT': 133, 'GCGAGTCCACCAATTG': 93, 'ATATAAATAAGCTATG': 51, 'TCTTTTCTTATTGAGG': 73, 'ATAAACGTTCCCCAAG': 107, 'ACCCGTGAACCAAAGC': 1, 'AATCGTACATCCCCAC': 123, 'CCACCAGCCATACGGC': 93, 'TGGACTCCACACCCCG': 76, 'TCAAGAAGCCTTGGAG': 150, 'TTTCTCCCGCATCTCG': 105, 'AATCAATACGACCTGT': 74, 'TGACAATACCCTACTC': 46, 'TTTCTCCCCCATCTCG': 1, 'CATTCACTACTCCCAA': 92, 'TCACCAGTGATCCCTT': 65, 'ACCACCGCCACACCCT': 44, 'GCTGCATTCCTGTGGC': 72, 'GTTCAGGACACACTTA': 70, 'CCAGCCTCACACTAGC': 107, 'ACGCGTGAACCAAAGC': 102, 'ACCAGTTCTCCCCGGG': 152, 'TCACGGCCACGAGTAA': 65, 'TAACACTGATGGCAAC': 76, 'GCATGTTATAAACTCC': 45, 'TCACCCCGACCAGCAG': 33, 'TTCCACCTCACCTTCT': 95, 'GATAGTCGAACACATG': 85, 'TGATGTAATGACAGGC': 74, 'GCACATAGGGAGCCAA': 58, 'TCTGGCAGGCCTCGCT': 122, 'GAACCCTAGAATCCCC': 69, 'CTAATTCAATCCTACG': 99, 'CTTACCATATTACAAA': 105, 'TGTACAGCAAAAGTAG': 69, 'TGACATCGCGAGACGG': 72, 'CCA

3. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [53]:
#initialize empty variables
maxcount = 0
mostcount_barcode = ''
 
 #loop through all barcodes
for barcode in HA_barcode_counts:

    #check to see if number of entries in dict for that specific barcode is longer than the longest recorded
    if HA_barcode_counts[barcode] > maxcount:

        #if this barcode has the most counts, re-adjust counter variable and assign that barcode to a new var
        maxcount = HA_barcode_counts[barcode]
        mostcount_barcode = barcode

print('the most common barcode is repeated', maxcount)
print('the barcode with the most repetitions is',mostcount_barcode)


the most common barcode is repeated 155
the barcode with the most repetitions is CCCGACCCGACATTAA
